In [1]:
import csv
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

In [3]:
class DataPrep:
    def __init__(self, folder='.', image_size=100, camera_h_fov=72.2, camera_h_res=3264, 
                 in_csv='data.csv', out_csv='final_data.csv', prefix='final_'):
        self.folder = folder
        self.image_size = image_size
        self.camera_fov_per_pixel = camera_h_fov/camera_h_res
        self.csv_in_path = os.path.join(folder, in_csv)
        self.csv_out_path = os.path.join(folder, out_csv)
        self.prefix = prefix
        self.headers = []
        self.in_data = []
        self.out_data = []
        self._readData()

    def _readData(self):
        self.in_data = []
        with open(self.csv_in_path, 'r') as file:
            csv_reader = csv.reader(file)
            self.headers = None
            for row in csv_reader:
                if self.headers is None:
                    self.headers = row
                else:
                    self.in_data.append(row)
    
    def _writeData(self):
        with open(self.csv_out_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(self.headers + ['SAMPLE_FOV'])
            writer.writerows(self.out_data)

    def _processImage(self, image_color):
        # convert the image to grayscale
        image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)

        # find the threshold value
        tv = self._findTheshold(image_gray)

        # threshold the image
        image_thresh = cv2.threshold(image_gray, tv, 255, cv2.THRESH_BINARY)[1]

        # get a mask for the feature
        mask = self._getFeatureMask(image_thresh)
        if mask is None:
            return None, None

        # normalize the intensity within the masked area and zero elsewhere
        image_norm = self._normalizeIntensity(image_gray, mask)
        if image_norm is None:
            return None, None

        # find the centroid of the feature and the bounding extents
        center_x, center_y, min_x, min_y, max_x, max_y = self._findCentroidAndBounds(mask)
        
        # calculate a size that can contain the centered image extents
        size = int(2 * max(center_y - min_y, max_y - center_y, center_x - min_x, max_x - center_x))
        # create a blank image
        image_centered = np.zeros((size, size), dtype=np.uint8)
        # crop to extents
        image_crop = image_norm[int(min_y):int(max_y),
                                int(min_x):int(max_x)]
        # determine the offsets to position the cropped image with the centroid in the center
        offset_x = int(min_x - (center_x - size/2))
        offset_y = int(min_y - (center_y - size/2))
        # paste into blank image
        image_centered[offset_y:offset_y+image_crop.shape[0], 
                       offset_x:offset_x+image_crop.shape[1]] = image_crop
        
        # resize the image
        image_resize = cv2.resize(image_centered, (self.image_size, self.image_size))
        # determine the actual fov for the sample
        sample_fov = self.camera_fov_per_pixel * size
        return image_resize, sample_fov
        
    def _findTheshold(self, image):
        # Create an elliptical, arc-shaped mask to sample around the edge of the plate.
        # Arc is inset two pixels from the edges and has a thickness of two,
        # so it should not touch the 0 value area.
        # Skips a 90 degree arc at the top of the image since vertical features of
        # the object may protrude into this area when tilt is close to zero.
        height = image.shape[0]
        width  = image.shape[1]
        mask = np.zeros((height, width), dtype=np.uint8)
        cv2.ellipse(mask, (width//2, height//2), (width//2 - 2, height//2 - 2), 0, -45, 180+45, 255, 2)
        
        # find max pixel values around edge of ellipse arc
        minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(image, mask=mask)
    
        # threshold value is max pixel value + 2
        return int(maxVal + 2)

    def _normalizeIntensity(self, image, mask):
        # normalize the image intensity
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(image, mask=mask)
        if max_val - min_val < 1:
            return None
        # real data values should be 1 to 255
        image_norm = np.clip((254/(max_val-min_val))*(image.astype(np.float32) - min_val) + 1, 0, 255).astype(np.uint8)
        # background will have value 0
        image_masked = cv2.bitwise_and(image_norm, image_norm, mask=mask)
        return image_masked

    def _getFeatureMask(self, image_thresh):
        # Find the largest contiguous closed region (ignore holes)
        # First find external contours only
        contours, hierarchy = cv2.findContours(image_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if len(contours) < 1:
            return None
        # Find the region with the longest perimeter, assume the rest is noise
        max_index = -1
        max_perim = -1
        for i in range(len(contours)):
            perim = len(contours[i])
            if perim > max_perim:
                max_perim = perim
                max_index = i
        # Create a mask for the feature
        mask = np.zeros(image_thresh.shape[:2], dtype=np.uint8)
        cv2.drawContours(mask, [contours[max_index]], -1, 255, -1)
        return mask

    def _findCentroidAndBounds(self, mask):
        # find the connected components in the mask to get the centroid and extents
        numLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
        if numLabels > 2:
            print("WARNING: more than one non-background region found")
        center_x = centroids[1][0]
        center_y = centroids[1][1]
        min_x = stats[1][0]
        min_y = stats[1][1]
        max_x = stats[1][0] + stats[1][2]
        max_y = stats[1][1] + stats[1][3]
        return center_x, center_y, min_x, min_y, max_x, max_y

    def processData(self):
        self.out_data = []
        for row in self.in_data:
            filename = row[0]
            path = os.path.join(self.folder, filename)
            image_color = cv2.imread(path)
            has_error = False
            try:
                image_proc, sample_fov = self._processImage(image_color)
            except:
                has_error = True
            if has_error:
                print(f'Skipping {filename}, error encountered during processing')
                continue
            if image_proc is None:
                print(f'Skipping {filename}, could not process')
                continue
            cv2.imwrite(os.path.join(self.folder, self.prefix + filename), image_proc)
            self.out_data.append([self.prefix + filename] + row[1:] + [sample_fov])
        self._writeData()

In [5]:
dp = DataPrep(folder=r'C:\Users\sarah\ScienceFairData')

In [7]:
dp.processData()